# Fill the shared array with data

In [2]:
import numpy as np
import skmap_bindings
import time
import tempfile
from pathlib import Path
from osgeo import gdal, gdal_array
import pandas as pd
import geopandas as gpd

def _model_input(tile, start_year = 2000, end_year = 2022, bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal'], base_url='http://192.168.49.30:8333'):
    prediction_layers = []

    for year in range(start_year, end_year + 1):
        for band in bands:
            prediction_layers += [
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0101_{year}0228_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0301_{year}0430_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0501_{year}0630_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0701_{year}0831_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0901_{year}1031_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}1101_{year}1231_go_epsg.4326_v20230908.tif'
            ]

    return prediction_layers

def _find_connected_indices(raster_files, str1, str2):
    list1 = []
    list2 = []    
    for i, elem1 in enumerate(raster_files):
        if str1 in elem1:
            for j, elem2 in enumerate(raster_files):
                if str2 in elem2 and elem1.split(str1)[-1] == elem2.split(str2)[-1]:
                    list1.append(i)
                    list2.append(j)
                    break                
    return list1, list2

def _raster_paths(df_features, ftype, tile = None, year = None):
    mask = (df_features['type'] == ftype)
    path_col = 'path'
    if ftype == 'landsat':
        df_features['ypath'] = df_features[mask]['path'].apply(lambda p: p.replace('{tile}', tile).replace('{year}', str(year)))
        path_col = 'ypath'
    ids_list = list(df_features[mask]['idx'])
    raster_files = list(df_features[mask][path_col])
    return raster_files, ids_list

def _get_static_layers_info(tiles_gpkg, features_csv, tile):
    tiles = gpd.read_file(tiles_gpkg)
    df_features = pd.read_csv(features_csv)
    min_x, _, _, max_y = tiles[tiles['TILE'] == tile].iloc[0].geometry.bounds
    static_files, _ = _raster_paths(df_features, 'static')
    gidal_ds = gdal.Open(static_files[0]) # It is assumed to be the same for all static layers
    gt = gidal_ds.GetGeoTransform()
    gti = gdal.InvGeoTransform(gt)
    x_off_s, y_off_s = gdal.ApplyGeoTransform(gti, min_x, max_y)
    x_off_s, y_off_s = int(x_off_s), int(y_off_s)
    return static_files, x_off_s, y_off_s

# Parameters setup

In [28]:
tile = '029E_51N'
conf_GDAL = {"CPL_VSIL_CURL_ALLOWED_EXTENSIONS": ".tif",
             "GDAL_HTTP_VERSION": "1.0"}
bands_list = [1,] # The first band is indexed as 1
x_size = 4000
y_size = 4000
n_threads = 96

tiles_gpkg = 'ard2_final_status.gpkg'
features_csv = './model/features.csv'
static_files, x_off_s, y_off_s = _get_static_layers_info(tiles_gpkg, features_csv, tile)
n_static = len(static_files)
idx_static = range(n_static)

x_off_d = 2
y_off_d = 2
dynamic_files = _model_input(tile, start_year = 2000, end_year = 2020, bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal'], base_url=f'http://{subnet}.30:8333')
subnet = '192.168.49'
hosts = [ f'{subnet}.{i}:8333' for i in range(30,43) ]
dynamic_files = [str(r).replace(f"{subnet}.30", f"{subnet}.{30 + int.from_bytes(Path(r).stem.encode(), 'little') % len(hosts)}") for r in dynamic_files]
n_dynamic = len(dynamic_files)
idx_dynamic = range(n_static, n_static+n_dynamic) # row indices of the array where each tiff contenct will go (starting from first element of file_urls)

list_red, list_nir = _find_connected_indices(static_files + dynamic_files, 'red_glad', 'nir_glad')
n_indices = len(list_red)
idx_indices = range(n_static + n_dynamic, n_static + n_dynamic + n_indices)

n_feat = n_static + n_dynamic + n_indices
data = np.empty((n_feat,x_size*y_size), dtype=np.float32, order='C')
print("Parameters setup done")

Parameters setup done


In [33]:
start = time.time()
skmap_bindings.readData(data, n_threads, static_files, idx_static, x_off_s, y_off_s, x_size, y_size, bands_list, conf_GDAL)
print(f"Reading static layers done in {time.time() - start:.2f} s")
print(data)
print(data.shape)

Reading static layers done in 3.01 s
[[  181.   181.   181. ...   146.   146.   146.]
 [14516. 14516. 14516. ... 14683. 14684. 14684.]
 [13564. 13564. 13564. ... 13497. 13497. 13497.]
 ...
 [  207.   208.   208. ...   207.   205.   208.]
 [  215.   219.   214. ...   215.   215.   215.]
 [  219.   219.   219. ...   216.   220.   221.]]
(1092, 16000000)


Warning 1: /vsicurl/http://192.168.49.41:8333/global/lcv/lcv_water.distance_glad.interanual.dynamic.classes_pxl_100m_0..0cm_1999..2021_v2021.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes


In [29]:
print("Starting")
start = time.time()
skmap_bindings.readData(data, n_threads, dynamic_files, idx_dynamic, x_off_d, y_off_d, x_size, y_size, bands_list, conf_GDAL, 255., np.nan)
print(f"Reading dynamic layers done in {time.time() - start:.2f} s")
print(data)
print(data.shape)

Starting
C++ reading done in 3.81 s
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1092, 16000000)


In [30]:
print("Starting")
start = time.time()
skmap_bindings.computeNormalizedDifference(data, n_threads,
                            list_nir, list_red, idx_indices,
                            0.004, 0.004, 125., 125.)
print(f"C++ ND done in {time.time() - start:.2f} s")
print(data)


Starting
C++ ND done in 0.82 s
[[  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [207. 208. 208. ... 207. 205. 208.]
 [215. 219. 214. ... 215. 215. 215.]
 [219. 219. 219. ... 216. 220. 221.]]
